In [2]:
# Import and intizialize
import pandas as pd
import json
from math import isnan


In [8]:
# Read datasets
migration_flow = pd.read_csv('migration_flows_bilateral_2015_clean.csv')
iso_codes = pd.read_csv('all.csv')

In [9]:
# Copy for splitting csv into two without changing the original variable
immigrants = migration_flow.copy()
emigrants = migration_flow.copy()

# Immigrants
immigrants = immigrants[immigrants.columns.drop(list(immigrants.filter(regex='EMIGRANTS')))]
immigrants = immigrants.iloc[:,:-8]

# Emigrants
emigrants = emigrants[emigrants.columns.drop(list(emigrants.filter(regex='IMMIGRANTS')))]
emigrants = emigrants.iloc[:,:-8] # drop last 8 columns since they are micro nations
emigrants.columns = emigrants.columns.str.replace("EMIGRANTS_FROM_","")

In [10]:
# Create a dictionary to rename column names
countries = emigrants.columns[2:] 

# Create a dictionary with country names as key and index as value
country_dict = {key: json.dumps(idx) for idx, key in enumerate(countries)}

# Replace column headers with numbers instead of country names
emigrants.rename(columns=country_dict, inplace=True) 

# convert dataframe to dictionary
flow = emigrants.iloc[:,2:].to_dict(orient='index')

# remove all nan values so dictionary is clean 
for i in flow: 
    for j in list(flow[i]):
        if isnan(flow[i][j]):
            del flow[i][j]
flower = json.dumps(flow)

In [12]:
# Add the flow dictionary values to column in dataframe where the key is row number
emigrants_15 = emigrants.iloc[:,1:2]
emigrants_15['flows'] = pd.Series(flow)
emigrants_15['short'] = emigrants_15['COUNTRY'].str[:4]

C:\Users\silva\AppData\Local\Temp\ipykernel_35356\3441384611.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emigrants_15['flows'] = pd.Series(flow)
C:\Users\silva\AppData\Local\Temp\ipykernel_35356\3441384611.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emigrants_15['short'] = emigrants_15['COUNTRY'].str[:4]


In [13]:
# Rearrange iso dataset for join with migration dataset
iso_codes['name'] = iso_codes['name'].str.upper()
iso = iso_codes.iloc[:,:2]
iso = iso.rename(columns = {'name':'COUNTRY', 'alpha-2':'ISO Code'})
iso['short'] = iso['COUNTRY'].str[:4]
iso = iso.iloc[:,1:3]

# Actual join via merge()
iso_emigrants = pd.merge(emigrants_15, iso, how="left", on=['short'])

In [14]:
# Manual data cleaning -> future check if values are correctly assigned!

# Cleaning ISO values
iso_emigrants.xs(39)['ISO Code'] = 'CV'
iso_emigrants.xs(40)['ISO Code'] = 'MQ' # As of now Martinique 
iso_emigrants.xs(44)['ISO Code'] = 'JE' # As of now Jersey
iso_emigrants.xs(53)['ISO Code'] = 'CI'
iso_emigrants.xs(59)['ISO Code'] = 'CD'
iso_emigrants.xs(74)['ISO Code'] = 'FO'
iso_emigrants.xs(125)['ISO Code'] = 'LA'
iso_emigrants.xs(162)['ISO Code'] = 'NA'
iso_emigrants.xs(193)['ISO Code'] = 'PF' # As of now French Polynesia
iso_emigrants.xs(197)['ISO Code'] = 'RE'
iso_emigrants.xs(323)['ISO Code'] = 'VA'

# We do not have values for North/South Korea



In [8]:
# Create a csv file to join in arcGis
iso_emigrants.to_csv('emi_15_flow.csv', index=False, encoding='utf-8')

In [2]:
# Import and intizialize
import pandas as pd
import json
from math import isnan

In [28]:
# Reintroduce the argis geoJson for further processing
f = open('i20.geoJson', 'r')
data = json.load(f)
f.close()

In [29]:
for i in range(len(data['features'])):
    string = data['features'][i]['properties']['flows']
    string = string.replace("'",'"') 
    data['features'][i]['properties']['flows'] = json.loads(string)

In [10]:
# Change flows to json format instead of string
for i in range(len(data['features'])):
    flow_json = iso_emigrants['flows'][i]
    data['features'][i]['properties']['flows'] = json.dumps(flow_json)

for i in range(len(data['features'])):
    flow_json = json.loads(data['features'][i]['properties']['flows'])
    data['features'][i]['properties']['flows'] = flow_json

NameError: name 'iso_emigrants' is not defined

In [30]:
# Create a centroid attribute where x,y are elements in list centroid 
for i in data['features']:
    i['geometry']['centroid'] = [i['properties']['X'],i['properties']['Y']]

# X is long, y is lat in dataset from arcgis

In [31]:
data['features'][5]['properties']

{'OBJECTID': 7,
 'Country': '6',
 'ISO Code': 'WS',
 'Affiliation': 'Samoa',
 'Affiliation ISO Code': 'WS',
 'X': -172.1594594644777,
 'Y': -13.758366525659419,
 'Shape_Length': 3.0196618946939964,
 'Shape_Area': 0.24551890561299944,
 'ID': 6,
 'COUNTRY': 'SAMOA',
 'flows': {'3': -15880.0,
  '11': -32754.0,
  '12': -6.0,
  '19': -6.0,
  '30': -5.0,
  '35': -171.0,
  '37': 0,
  '47': -9.0,
  '54': 0.0,
  '56': -22.0,
  '70': -302.0,
  '71': -2.0,
  '72': -18.0,
  '81': -455.0,
  '87': -25.0,
  '92': -1.0,
  '98': -13.0,
  '101': -63.0,
  '117': 0.0,
  '142': -25.0,
  '144': -57947.0,
  '148': -44.0,
  '151': -3.0,
  '176': 4021.0,
  '186': 0.0,
  '196': -27.0,
  '197': -15.0,
  '204': -195.0,
  '205': -217.0,
  '211': -2.0,
  '215': -319.0,
  '216': -27186.0},
 'short': 'SAMOA'}

In [32]:
# Write Json File
json_object = json.dumps(data)
with open('i20.json', 'w') as outfile:
    outfile.write(json_object)
